<a href="https://colab.research.google.com/github/abolfazlaghdaee/LLM_journey/blob/main/Fine_tuning_a_model_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):

  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_dataset_train = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,

)

tf_dataset_validation = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


**Training**

In [3]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.fit(tf_dataset_train, validation_data = tf_dataset_validation, verbose=1)

459/459 [==============================] - 134s 173ms/step - loss: 0.6886 - accuracy: 0.6243 - val_loss: 0.6334 - val_accuracy: 0.6838


**Improving training performance**

If you try the above code, it certainly runs, but you’ll find that the loss declines only slowly or sporadically. The primary cause is the learning rate. As with the loss, when we pass Keras the name of an optimizer as a string, Keras initializes that optimizer with default values for all parameters, including learning rate. From long experience, though, we know that transformer models benefit from a much lower learning rate than the default for Adam, which is 1e-3, also written as 10 to the power of -3, or 0.001. 5e-5 (0.00005), which is some twenty times lower, is a much better starting point.


In addition to lowering the learning rate, we have a second trick up our sleeve: We can slowly reduce the learning rate over the course of training. In the literature, you will sometimes see this referred to as decaying or annealing the learning rate. In Keras, the best way to do this is to use a learning rate scheduler. A good one to use is PolynomialDecay — despite the name, with default settings it simply linearly decays the learning rate from the initial value to the final value over the course of training, which is exactly what we want. In order to use a scheduler correctly, though, we need to tell it how long training is going to be. We compute that as num_train_steps below.


In [5]:
import tensorflow
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3



In [6]:
num_steps = len(tf_dataset_train) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.0,
    decay_steps=num_steps
)

import keras
from tensorflow.python.keras.optimizers import adam_v2



In [7]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, create_optimizer
from tensorflow.keras.optimizers.schedules import PolynomialDecay

# ... your existing code for loading data and defining lr_scheduler ...

# Load the model
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Define the loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Create the optimizer using create_optimizer
num_train_steps = len(tf_dataset_train) * num_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=5e-5,  # Your initial learning rate
    num_warmup_steps=0,  # Number of warmup steps (adjust as needed)
    num_train_steps=num_train_steps,
)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# ... your existing code for training ...

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model.fit(tf_dataset_train,
          validation_data= tf_dataset_validation,
          epochs=3 )

Epoch 1/3
459/459 [==============================] - 116s 168ms/step - loss: 0.5524 - accuracy: 0.7271 - val_loss: 0.3854 - val_accuracy: 0.8186
Epoch 2/3
459/459 [==============================] - 79s 172ms/step - loss: 0.3062 - accuracy: 0.8746 - val_loss: 0.3307 - val_accuracy: 0.8431
Epoch 3/3
459/459 [==============================] - 80s 174ms/step - loss: 0.0965 - accuracy: 0.9708 - val_loss: 0.4009 - val_accuracy: 0.8554


**Model predictions**

In [19]:
pred = model.predict(tf_dataset_validation)
class_pred = np.argmax(pred.logits, axis = 1)

51/51 [==============================] - 3s 61ms/step


In [23]:
print(class_pred.shape)
print(pred.logits.shape)

(408,)
(408, 2)


In [26]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [27]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_pred, references=raw_datasets["validation"]["label"])

{'accuracy': 0.8553921568627451, 'f1': 0.9005059021922428}

end!